In [ ]:
# no paired file. only biological files

In [1]:
import os
from parse_soft import ParseSoft
from search_soft import SearchSoft
from enrich_soft import EnrichSoft
from label_data import LabelData

%load_ext autoreload
%autoreload 2

In [2]:
# step 1: set defaults
geo = "GSE148673"
soft_dir = '/home/yuan/data'
meta_dir = '../data'
download_dir = '/home/yuan/rawdata/SRR'

In [10]:
# step 2: parse soft, and enrich

# analyze soft file
parser = ParseSoft(soft_dir)
soft_path = parser.soft_local_path(geo)
print('Retrieve meta data from soft file: ', soft_path)
softer = SearchSoft(soft_path)
data = softer.filter_data(softer.parse_rows)

# save to json
LabelData(meta_dir).save(data)

Retrieve meta data from soft file:  /home/yuan/data/ftp.ncbi.nlm.nih.gov/geo/series/GSE148nnn/GSE148673/soft/GSE148673_family.soft.gz
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE148nnn/GSE148673/metadata.json


In [11]:
# step 3a: enrich data
data = LabelData(meta_dir).get_meta(geo)
enriched = EnrichSoft(data)()
LabelData(meta_dir).save(enriched)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE148nnn/GSE148673/metadata.json


In [12]:
# step 3b: label samples manually
'''
46,501 single cells from 21 tumors
triple-negative breast cancer TNBC
pancreatic ductal adenocarcinomas 
anaplastic thyroid cancer, ATC
invasive ductal carcinoma, IDC 
glioblastoma
'''
from label_sample import LabelSample

def collator(sample):
    c = LabelSample(sample)
    c._protocol('scrna-seq', "10x3v2")
    if c.title.startswith('TNBC'):
        c._disease_patient('triple-negative breast cancer')
        c._update('tissue', 'breast')
    elif c.title.startswith('DCIS'):
        c._disease_patient('ductal carcinoma in situ, breast cancer')
        c._update('tissue', 'breast')
    elif c.title.startswith('IDC'):
        c._disease_patient('invasive ductal carcinoma, breast cancer')
        c._update('tissue', 'breast')
    elif c.title.startswith('ATC'):
        c._disease_patient('anaplastic thyroid cancer')
        c._update('tissue', 'thyroid')

data = LabelData(meta_dir).get_meta(geo)
for sample_id, sample in data['samples'].items():
    collator(sample)
LabelData(meta_dir).save(data)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE148nnn/GSE148673/metadata.json


In [15]:
for sample_id, sample in data['samples'].items():
    print(sample['sample_title'], sample['labels'])

DCIS1 {'tissue': 'breast', 'disease': 'ductal carcinoma in situ, breast cancer', 'group': 'patient'}
TNBC1 {'tissue': 'breast', 'disease': 'triple-negative breast cancer', 'group': 'patient'}
TNBC2 {'tissue': 'breast', 'disease': 'triple-negative breast cancer', 'group': 'patient'}
TNBC3 {'tissue': 'breast', 'disease': 'triple-negative breast cancer', 'group': 'patient'}
TNBC4 {'tissue': 'breast', 'disease': 'triple-negative breast cancer', 'group': 'patient'}
TNBC5 {'tissue': 'breast', 'disease': 'triple-negative breast cancer', 'group': 'patient'}
ATC1 {'tissue': 'thyroid', 'disease': 'anaplastic thyroid cancer', 'group': 'patient'}
ATC2 {'tissue': 'thyroid', 'disease': 'anaplastic thyroid cancer', 'group': 'patient'}
ATC3 {'tissue': 'thyroid', 'disease': 'anaplastic thyroid cancer', 'group': 'patient'}
ATC4 {'tissue': 'thyroid', 'disease': 'anaplastic thyroid cancer', 'group': 'patient'}
ATC5 {'tissue': 'thyroid', 'disease': 'anaplastic thyroid cancer', 'group': 'patient'}
IDC1 {'ti

In [16]:
# step 4a: load some data
from utils import Utils

# data are determined previously
samn_srr = Utils.from_json('../results/samn_srr.json')
print(len(samn_srr))

47


In [17]:
# step 4b: Given BioSample, parse SRR accessions into samples
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr(data, samn_srr)
LabelData(meta_dir).save(enriched)

biosamples = 13, SRR accessions = 13.
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE148nnn/GSE148673/metadata.json


In [19]:
# step 5: merge srr_urls into data

# sample.<sample_id>.SRR.<SRR accession>."ftp.sra.ebi.ac.uk"
file_name = 'srr_fastq_urls_simple.json'
urls_json = os.path.join('../results', file_name)
urls = Utils.from_json(urls_json)

# parse urls
data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr_urls(data, urls)
LabelData(meta_dir).save(enriched)

{'srr': 13, 'available': 0, 'unknown': 13, 'updated': 0}
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE148nnn/GSE148673/metadata.json


In [20]:
# step 6a: download raw data using biosample accession using nextflow
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_biosample(sample_iter)
# run the bash script showed here
# Outcome: nf download failed

Number of biosamples:  13
Number of biosamples with SRR:  13
bash /home/yuan/rawdata/SRR/GSE148673/fetch1.sh


In [23]:
#Step 6b: collect all SRR without local_fastq then execute bash run2.sh
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_ebi_srr(sample_iter)
# run the bash script showed here

Number of SRR:  13
bash /home/yuan/rawdata/SRR/GSE148673/fetch2.sh


In [24]:
#Step 6c: collect all SRR without local_fastq then execute bash run3.sh
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_srr(sample_iter)
# run the bash script showed here

Number of SRR:  13
bash /home/yuan/rawdata/SRR/GSE148673/fetch3.sh


In [ ]:
# step 7: parse path of local fastq.gz with SRR in GEO data
# Note: run the process multiple times to make sure all SRRs are accessed
# print all samples if not any local *.fastq.gz is parsed
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
data, unparsing = ParseSra.parse_local_fastq(data, download_dir)
if unparsing:
    print(geo, len(unparsing), unparsing[:6])
LabelData(meta_dir).save(data)